In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import os
import tarfile
import glob

Matplotlib created a temporary config/cache directory at /scratch/slurm-job.828179/matplotlib-l72gzj16 because the default path (/cluster/customapps/biomed/grlab/users/prelotla/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [2]:
tar_file_OHSU = '/cluster/work/grlab/projects/projects2020_OHSU/share_OHUS_PNLL/Mar9-2023_OHSU_filterresults_realign_all/OHSU_March9-2023_filter_results.tar.gz'

In [3]:
base_eth = '/cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/CANCER_eth/commit_c4dd02c_conf2_Frame_cap0_runs/TCGA_Breast_1102/filtering_samples/chosen_filters_06March_order'

In [21]:
one_file_eth = '/cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/CANCER_eth/commit_c4dd02c_conf2_Frame_cap0_runs/TCGA_Breast_1102/filtering_samples/chosen_filters_06March_order/G_TCGA-AO-A0JM-01A-21R-A056-07_SampleLim0.0CohortLimNoneAcrossNone_FiltNormalsGtexCohortCohortlim0.0Across1.tsv.gz'

In [ ]:
# Reads > 0 or 1 sample any read
# Any read would be > 0 

In [5]:
# ETH Names
eth_all = glob.glob(os.path.join(base_eth, 'G*'))

In [6]:
# OHSU Names
with tarfile.open(tar_file_OHSU, "r:*") as tar:
    ohsu_all = tar.getnames()

In [7]:
# Get file pairs
file_pair = {}
for idx_eth, eth in enumerate(eth_all):
    pattern = os.path.basename(eth).replace('G_', '').replace('.gz', '').replace('.0CohortLim','CohortLim')
    for idx_ohsu, ohsu in enumerate(ohsu_all):
        if pattern in ohsu:
            file_pair[eth] = ohsu

            

In [27]:
restrict = 'TCGA-AO-A0JM-01A-21R-A056-07'
df = {'sample' : [], 
      'filter_foreground' : [], 
      'filter_background' : [], 
      'size_ohsu' : [], 
      'size_eth' : [], 
      'size_intersection' : [], 
      'size_ohsu\eth' : [], 
      'size_eth\ohsu' : []}
with tarfile.open(tar_file_OHSU, "r:*") as tar: #OHSU
    for eth, ohsu in file_pair.items(): # ETH
        if (not restrict) or restrict in eth: #Restrict to category of interest
            try:
                df_ohsu = pd.read_csv(tar.extractfile(ohsu), header=0, sep="\t")
                df_eth = pd.read_csv(eth, sep="\t", usecols = ['kmer'])
                df_eth = set(df_eth['kmer'])
                df_ohsu = set(df_ohsu['kmer'])
                name = os.path.basename(ohsu).replace('.tsv', '').split('_')
                df['sample'].append(name[1])
                df['filter_foreground'].append(name[2])
                df['filter_background'].append(name[3])
                df['size_ohsu'].append(len(df_ohsu))
                df['size_eth'].append(len(df_eth))
                df['size_ohsu\eth'].append(len(df_ohsu.difference(df_eth)))
                df['size_eth\ohsu'].append(len(df_eth.difference(df_ohsu)))
                df['size_intersection'].append(len(df_ohsu.intersection(df_eth)))
            except:
                continue


In [28]:
df

{'sample': ['TCGA-AO-A0JM-01A-21R-A056-07',
  'TCGA-AO-A0JM-01A-21R-A056-07',
  'TCGA-AO-A0JM-01A-21R-A056-07',
  'TCGA-AO-A0JM-01A-21R-A056-07',
  'TCGA-AO-A0JM-01A-21R-A056-07',
  'TCGA-AO-A0JM-01A-21R-A056-07',
  'TCGA-AO-A0JM-01A-21R-A056-07',
  'TCGA-AO-A0JM-01A-21R-A056-07',
  'TCGA-AO-A0JM-01A-21R-A056-07',
  'TCGA-AO-A0JM-01A-21R-A056-07'],
 'filter_foreground': ['SampleLim0CohortLim0.0Across1',
  'SampleLim0CohortLim2.0Across5',
  'SampleLim0CohortLim2.0Across1',
  'SampleLim0CohortLim2.0Across1',
  'SampleLim0CohortLim0.0Across5',
  'SampleLim0CohortLimNoneAcrossNone',
  'SampleLim0CohortLim2.0Across5',
  'SampleLim0CohortLim0.0Across5',
  'SampleLim0CohortLimNoneAcrossNone',
  'SampleLim0CohortLim0.0Across1'],
 'filter_background': ['FiltNormalsGtexCohortCohortlim3.0Across10',
  'FiltNormalsGtexCohortCohortlim3.0Across2',
  'FiltNormalsGtexCohortCohortlim3.0Across2',
  'FiltNormalsGtexCohortCohortlim3.0Across10',
  'FiltNormalsGtexCohortCohortlim3.0Across2',
  'FiltNormalsGt

In [43]:
foo = set()
for i in eth_all:
    if 'FiltNormalsGtexC' in i:
        if 'TCGA-AO-A0JM-01A-21R-A056-07' in i:
            foo.add(os.path.basename(i).split('_')[2])
            print(i)

/cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/CANCER_eth/commit_c4dd02c_conf2_Frame_cap0_runs/TCGA_Breast_1102/filtering_samples/chosen_filters_06March_order/G_TCGA-AO-A0JM-01A-21R-A056-07_SampleLim0.0CohortLim0.0Across1_FiltNormalsGtexCohortCohortlim3.0Across10.tsv.gz
/cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/CANCER_eth/commit_c4dd02c_conf2_Frame_cap0_runs/TCGA_Breast_1102/filtering_samples/chosen_filters_06March_order/G_TCGA-AO-A0JM-01A-21R-A056-07_SampleLim0.0CohortLim2.0Across5_FiltNormalsGtexCohortCohortlim3.0Across2.tsv.gz
/cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/CANCER_eth/commit_c4dd02c_conf2_Frame_cap0_runs/TCGA_Breast_1102/filtering_samples/chosen_filters_06March_order/G_TCGA-AO-A0JM-01A-21R-A056-07_SampleLim0.0CohortLim0.0Across5_FiltNormalsGtexCohortCohortlim0.0Across1.tsv.gz
/cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/CANCER_eth/commit_c4dd02c_conf2_Frame_cap0_runs/TCGA_Breast_11

In [44]:
foo

{'SampleLim0.0CohortLim0.0Across1',
 'SampleLim0.0CohortLim0.0Across5',
 'SampleLim0.0CohortLim2.0Across1',
 'SampleLim0.0CohortLim2.0Across5',
 'SampleLim0.0CohortLimNoneAcrossNone'}

In [49]:
for i in ohsu_all:
    if ('TCGA-AO-A0JM-01A-21R-A056-07' in i) \
    and('FiltNormalsGtexC' in i) \
    and ('SampleLim0CohortLimNoneAcrossNone' in i) \
    and ('FiltNormalsGtexCohortCohortlim0.0Across0' in i):
        print(i)

output/kmer_files/J_TCGA-AO-A0JM-01A-21R-A056-07_SampleLim0CohortLimNoneAcrossNone_FiltNormalsGtexCohortCohortlim0.0Across0.tsv


In [ ]:
# FiltNormalsGtexCohortCohortlim0.0Across0.tsv
